## Hyperexponential Case

Throughout this document, the following packages are required:

In [9]:
import numpy as np
import scipy
import math
from scipy.stats import binom, erlang, poisson
from scipy.optimize import minimize
from functools import lru_cache

### Plot Phase-Type Fit

In [10]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [11]:
def SCV_to_params(SCV):
    
    # weighted Erlang case
    if SCV <= 1:
        K = math.floor(1/SCV)
        p = ((K + 1) * SCV - math.sqrt((K + 1) * (1 - K * SCV))) / (SCV + 1)
        mu = K + (1 - p) * (K + 1)
    
        return K, p, mu
    
    # hyperexponential case
    else:
        p = 0.5 * (1 + np.sqrt((SCV - 1) / (SCV + 1)))
        mu = 1 # 1 / mean
        mu1 = 2 * p * mu
        mu2 = 2 * (1 - p) * mu
        
        return p, mu1, mu2

In [13]:
for i in range(81):
    SCV = 1 + 0.1 * i
    print(round(SCV,2),SCV_to_params(SCV))

1.0 (1, 1.0, 1.0)
1.1 (0.6091089451179963, 1.2182178902359926, 0.7817821097640074)
1.2 (0.6507556722888818, 1.3015113445777635, 0.6984886554222365)
1.3 (0.6805787796286538, 1.3611575592573075, 0.6388424407426925)
1.4 (0.7041241452319315, 1.408248290463863, 0.5917517095361371)
1.5 (0.7236067977499789, 1.4472135954999579, 0.5527864045000421)
1.6 (0.7401922307076307, 1.4803844614152615, 0.5196155385847385)
1.7 (0.7545875386086578, 1.5091750772173156, 0.4908249227826844)
1.8 (0.7672612419124244, 1.5345224838248488, 0.4654775161751512)
1.9 (0.7785430072655778, 1.5570860145311556, 0.4429139854688444)
2.0 (0.7886751345948129, 1.5773502691896257, 0.42264973081037427)
2.1 (0.7978416985906354, 1.5956833971812707, 0.4043166028187293)
2.2 (0.8061862178478973, 1.6123724356957947, 0.3876275643042053)
2.3 (0.8138229572304239, 1.6276459144608477, 0.3723540855391523)
2.4 (0.8208444739598739, 1.6416889479197478, 0.35831105208025216)
2.5 (0.8273268353539885, 1.654653670707977, 0.3453463292920229)
2.6 (0.

In [12]:
def density_WE(x, K, p, mu):
    return p * erlang.pdf(x, K, scale=1/mu) + (1 - p) * erlang.pdf(x, K+1, scale=1/mu)

def density_HE(x, p, mu1, mu2):
    return p * mu1 * np.exp(-mu1 * x) + (1 - p) * mu2 * np.exp(-mu2 * x)

In [9]:
x = np.linspace(0,4,1001)

def plot_f(SCV=1):
    
    if SCV <= 1:
        K, p, mu = SCV_to_params(SCV)
        f_x = density_WE(x, K, p, mu)
        title = f'SCV = {SCV}\n p = {p:.2f}, $K$ = {K}, $\mu$ = {mu:.2f}'
    else:
        p, mu1, mu2 = SCV_to_params(SCV)
        f_x = density_HE(x, p, mu1, mu2)
        title = f'SCV = {SCV}\n p = {p:.2f}, $\mu_1$ = {mu1:.2f}, $\mu_2$ = {mu2:.2f}'
    
    plt.plot(x,f_x)
    plt.title(title)
    plt.xlabel('$x$')
    plt.ylabel('density')
    plt.ylim(0,2)


In [10]:
interact(plot_f, SCV=(0.01,2,0.01));

interactive(children=(FloatSlider(value=1.0, description='SCV', max=2.0, min=0.01, step=0.01), Output()), _dom…

The recursion of the dynamic program is given as follows. For $i=1,\dots,n-1$, $k=1,\dots,i$, and $m\in\mathbb{N}_0$,

\begin{align*}
\xi_i(k,m) &= \inf_{t\in \mathbb{N}_0}
\Big(
\omega \bar{f}^{\circ}_{k,m\Delta}(t\Delta) + (1-\omega)\bar{h}^{\circ}_{k,m\Delta} +
\sum_{\ell=2}^{k}\sum_{j=0}^{t}\bar{q}_{k\ell,mj}(t)\xi_{i+1}(\ell,j) +
P^{\downarrow}_{k,m\Delta}(t\Delta)\xi_{i+1}(1,0) +
P^{\uparrow}_{k,m\Delta}(t\Delta)\xi_{i+1}(k+1,m+t)
\Big),
\end{align*}

whereas, for $k=1,\dots,n$ and $m\in \mathbb{N}_0$,

\begin{align*}
\xi_n(k,m) = (1-\omega)\bar{h}^{\circ}_{k,m\Delta}.
\end{align*}

We will implement this dynamic program step by step. First, we implement all functions in the equation above.

Our formulas rely heavily on the survival function $\mathbb{P}(B>t)$ and $\gamma_z(t) = \mathbb{P}(Z_t = z\mid B>t)$:

In [49]:
@lru_cache(maxsize=128)
def B_sf(t):
    """The survival function P(B > t)."""
    return p * np.exp(-mu1 * t) + (1 - p) * np.exp(-mu2 * t)

@lru_cache(maxsize=128)
def gamma(z, t):
    """Computes P(Z_t = z | B > t)."""
    
    gamma_circ = B_sf(t)
    
    if z == 1:
        return p * np.exp(-mu1 * t) / gamma_circ
    elif z == 2:
        return (1 - p) * np.exp(-mu2 * t) / gamma_circ

Next, we implement $\bar{f}^{\circ}_{k,u}(t)$, which depends on $\bar{f}_{k,z}(t)$:

In [50]:
@lru_cache(maxsize=128)
def f_bar(k,z,t):
    
    if z == 1:
        return sum([binom.pmf(m, k-1, p) * sigma(t, m+1, k-1-m) for m in range(k)])
    elif z == 2:
        return sum([binom.pmf(m, k-1, p) * sigma(t, m, k-m) for m in range(k)])

@lru_cache(maxsize=128)
def f_circ(k, u, t):
    return gamma(1, u) * f_bar(k, 1, t) + gamma(2, u) * f_bar(k, 2, t)

In here, we need to evaluate the object $\sigma_{t}[m,k]$, which depends on $\rho_{t}[m,k]$:

In [75]:
@lru_cache(maxsize=512)
def sigma(t,m,k):
    
    return (t - k / mu2) * erlang.cdf(t, m, scale=1/mu1) - (m / mu1) * erlang.cdf(t, m+1, mu1) + \
            (mu1 / mu2) * sum([(k-i) * rho_t(t, m-1, i) for i in range(k)])

@lru_cache(maxsize=512)
def rho_t(t,m,k):
    
    if not k:
        return np.exp(-mu2 * t) * (mu1 ** m) / ((mu1 - mu2) ** (m + 1)) * erlang.cdf(t, m+1, scale=1/(mu1 - mu2))
    elif not m:
        return np.exp(-mu1 * t) * (mu2 ** k) / ((mu1 - mu2) ** (k + 1)) * erlang.cdf(t, k+1, scale=1/(mu1 - mu2))
    else:
        return (mu1 * rho(t,a,m-1,k) - mu2 * rho(t,a,m,k-1)) / (mu1 - mu2)
    

@lru_cache(maxsize=512)
def rho(t,a,m,k):
    
    if not k:
        return np.exp(-mu2 * t) * (mu1 ** m) / ((mu1 - mu2) ** (m + 1)) * erlang.cdf(a, m+1, scale=1/(mu1 - mu2))
    elif not m:
        return np.exp(-mu1 * t) * (mu2 ** k) / ((mu1 - mu2) ** (k + 1)) * \
                    (erlang.cdf(t, k+1, scale=1/(mu1 - mu2)) - erlang.cdf(t-a, k+1, scale=1/(mu1 - mu2)))
    else:
        return (mu1 * rho(t,a,m-1,k) - mu2 * rho(t,a,m,k-1) - r(t,a,m,k)) / (mu1 - mu2)


@lru_cache(maxsize=512)
def r(t,s,m,k):
    return poisson.pmf(m,mu1*s) * poisson.pmf(k,t-s)

We do the same for $\bar{h}^{\circ}_{k,u}(t)$, which only depends on $\bar{h}_{k,z}$:

In [ ]:
@lru_cache(maxsize=128)
def h_bar(k, z):

    if k == 1:
        return 0
    elif z <= K:
        return ((k - 1) * (K + 1 - p) + 1 - z) / mu
    elif z == K + 1:
        return ((k - 2) * (K + 1 - p) + 1) / mu

# @lru_cache(maxsize=128)
# def h_circ(k, u):
#     return gamma(1, u) * h_bar() sum([gamma(z, u) * h_bar(k, z) for z in range(1, K+2)])

The next objective is to implement $\bar{q}_{k\ell,mj}(t)$. This function depends on $q_{k\ell,z,v}(t)$, which depends on $\psi_{vt}[k,\ell]$: TODO

In [76]:
# TODO

In [5]:
poisson.pmf(3,0)

0.0

Finally, we implement the remaining transition probabilities $P^{\uparrow}_{k,u}(t)$ and $P^{\downarrow}_{k,u}(t)$:

In [83]:
# @lru_cache(maxsize=128)
def P_up(k, u, t):
    """Computes P(N_t- = k | N_0 = k, B_0 = u)."""
    return B_sf(u + t) / B_sf(u)

@lru_cache(maxsize=128)
def P_down(k, u, t):
    """Computes P(N_t- = 0 | N_0 = k, B_0 = u)."""
    return sum([binom.pmf(m, k, p) * Psi(t, m, k-m) for m in range(k+1)])

@lru_cache(maxsize=128)
def Psi(t, m, k):
    return erlang.cdf(t, m, scale=1/mu1) - mu1 * sum([rho_t(t, m-1, i) for i in range(k)])

In [4]:
erlang.cdf(0,1,1)

0.0